## Data Visualization Course | VNU-HCM University of Science.
### Final Project
#### **STEP 02 - DATA Preprocessing**

In [ ]:
import sys
import os
shared_path = os.path.abspath(os.path.join(os.getcwd(), '..', 'Shared'))
sys.path.append(shared_path)

In [ ]:
from Libraries import *
from Shared_Functions import *

In [ ]:
dataset_source_path = getGranDir() / 'Dataset'
pre_process_dataset_path = dataset_source_path / '2021.csv'
raw_df = pd.read_csv(pre_process_dataset_path, index_col=0)
raw_df.head()

**1. Kiểm tra kiểu dữ liệu và số lượng hàng, cột**

In [ ]:
print(raw_df.dtypes)
raw_df.shape

**Nhận xét:** Các cột đã có kiểu dữ liệu phù hợp với ý nghĩa.

**2. Kiểm tra điều kiện tồn tại của các cột.**

**2.1 Kiểm tra cột ID: các ID này phải là DUY NHẤT, và phải lớn hơn 0.**

In [ ]:
# ID
ID_col = raw_df['id']
less_than_0 = ID_col[ID_col < 0]
duplicate_values = ID_col[ID_col.duplicated(keep=False)]
print('Less than zero: ')
print(less_than_0)
print('Values is not unique: ')
print(duplicate_values)

**Nhận xét:** Như vậy không có giá trị nào bé hơn không và không có giá trị nào bị trùng lập.

**Kết luận:** Vì không có trùng lặp cho nên không có hàng nào trong dataset sẽ bị trùng lập.

**2.2 Kiểm tra cột Year: Year phải lớn hơn 0 và phải bằng 2022.**

In [ ]:
#Year
Year_col = raw_df['year']
less_than_0 = Year_col[Year_col < 0]
is_not_2022 = Year_col[Year_col != 2022]
print('Less than zero: ')
print(less_than_0)
print('Value must be 2022: ')
print(is_not_2022)

**Nhận xét:** Như vậy không có giá trị nào khác 2022 và bé hơn 0.
**Kết luận:** Vì không có giá trị nào khác 2022 cho nên việc phân tích cột này sẽ không có ý nghĩa. Do đó ta sẽ bỏ cột này luôn.

In [ ]:
raw_df = raw_df.drop(columns='year')
raw_df.head()

**2.3 Kiểm tra các cột điểm - score: Score phải >= 0 và <= 10 hoặc bằng NaN.**

In [ ]:
score_df = raw_df.select_dtypes(include='float64')
score_df.head()
for col in score_df.columns:
    less_than_0 = score_df[score_df[col] < 0]
    greater_than_10 = score_df[score_df[col] > 10]
    
    if len(less_than_0.index.tolist()):
        print(f'Column: {col}')
        print('Less than zero (indices): ')
        print(less_than_0.index.tolist())
    
    if len(greater_than_10.index.tolist()):
        print(f'Column: {col}')
        print('Greater than ten (indices): ')
        print(greater_than_10.index.tolist())

**Nhận xét:** Không có output được in ra, tức nghĩa là các cột đều đã thõa mãn điều kiện.

**2.4 Kiểm tra cột có kiểu dữ liệu không phải dữ liệu kiểu số**

Ở đây ta chỉ cần tìm ra các trường DUY NHẤT.

In [ ]:
non_numerical = raw_df['foreign_language_type']
unique_values = non_numerical.unique()
unique_values

**3. Kiểm tra các thiếu khuyết trong dữ liệu:**

In [ ]:
def calcMissingRatio(df):
    missing_ratio = {}
    for key, values in df.items():
        missing_count = values.isna().sum()
        total_count = len(values)
        missing_ratio[key] = round(( missing_count / total_count )* 100, 1) if total_count != 0 else None
    return missing_ratio

In [ ]:
missing_ratio = calcMissingRatio(raw_df)
for key, values in missing_ratio.items():
    print(f'Column: {key} - Missing Ratio: {values}')

**Nhận xét:**
1. Ta thấy các cột như: `russian_score`, `german_score`, `french_score`, `korean_score`, `japanese_score`, `chinese_score` đều có tỉ lệ thiếu khuyết lên đến hơn 99.9%. Do đó sẽ không có quá nhiều ý nghĩa phân tích. 
2. Các cột như `id` và `year` có tỉ lệ thiếu khuyết là 0.0%.

**Kết luận:** 
1. Ta sẽ chỉ xét các môn thi chính thức như toán - lý - hóa - sinh - văn - anh văn - sử - địa - GDCD. 
2. Do đó ta sẽ tiến hành loại bỏ các cột không có ý nghĩa phân tích: `russian_score`, `german_score`, `french_score`, `korean_score`, `japanese_score`, `chinese_score`, `foreign_language_score`.
3. Vì do chúng ta đã xóa các cột ngoại ngữ khác vì tỉ lệ thiếu khuyết cao thì ta sẽ xóa luôn cột định nghĩa ký hiệu cho các loại ngoại ngữ : `foreign_language_type`

In [ ]:
raw_df = raw_df.drop(columns = ['russian_score', 'german_score', 'french_score', 'korean_score', 'japanese_score', 'chinese_score', 'foreign_language_score', 'foreign_language_type'])
raw_df.head()

**4. Có hai khối thi là Khoa học tự nhiên và Khoa học xã hội, khi ta thi KHTN ( lý - hóa - sinh ) thì sẽ không thi các môn KHXH ( sử - địa - GDCD ).**

Chính vì vậy mà ta sẽ không tiến hành điền khuyết NaN vào trong các trường dữ liệu này. Mà ta sẽ phân tích dựa theo hai tổ hợp môn này.

**5. Sắp xếp các kết quả hàng theo id.**

In [ ]:
raw_df.sort_values(by='id', ascending=True, inplace=True)
raw_df.head()

**6. Tiến hành ghi tập dữ liệu sau khi xử lý vào file csv.**

In [ ]:
processed_dataset_path = dataset_source_path / 'THPTQG_2022_processed.csv'
raw_df.to_csv(processed_dataset_path, index=False)